## Reloading Julia Sim

### Set Up Model

In [1]:

OUTPUT_FOLDER_PRE = "C:/Users/Yan/OneDrive/Desktop/UNI/Honours/ml-for-da-yanny-honours-2025/"
LAYER_FOLDER = "one_layer_up_to_training"
OG_REANALYSIS_NC = "one_layer_primitive_wet/output.nc"
REANALYSIS_NPZ = "reanalysis/speedy_1layer_u_v_reanalysis.npz"

"reanalysis/speedy_1layer_u_v_reanalysis.npz"

In [ ]:
using SpeedyWeather
using NCDatasets

spectral_grid = SpectralGrid(NF = Float64, trunc = 31, nlayers=1, Grid=FullGaussianGrid{Float64})
output = NetCDFOutput(spectral_grid, path=OUTPUT_FOLDER_PRE*LAYER_FOLDER)

@kwdef struct TempOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "temp"
    unit::String = "K"
    long_name::String = "temperature"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct SurPresOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "spres"
    unit::String = "Pa"
    long_name::String = "surface pressure"
    dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
end

@kwdef struct DivOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "div"
    unit::String = "s^-1"
    long_name::String = "divergence"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct HumidOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "humid"
    unit::String = ""
    long_name::String = "specific humidity"
    dims_xyzt::NTuple{4, Bool} = (true, true, true, true)
end

@kwdef struct PrecipOutput <: SpeedyWeather.AbstractOutputVariable
    name::String = "precip"
    unit::String = ""
    long_name::String = "precipitation"
    dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
end
#@kwdef struct PresOutput <: SpeedyWeather.AbstractOutputVariable
   # name::String = "log pres"
   # unit::String = "Pa"
   # long_name::String = "pressure"
   # dims_xyzt::NTuple{4, Bool} = (true, true, false, true)
#end

add!(output, TempOutput(), SurPresOutput(), DivOutput(), HumidOutput(), SpeedyWeather.PrecipitationOutput()...)
SpeedyWeather.path(::TempOutput, simulation) =
    simulation.diagnostic_variables.grid.temp_grid
SpeedyWeather.path(::SurPresOutput, simulation) =
    simulation.diagnostic_variables.grid.pres_grid
SpeedyWeather.path(::DivOutput, simulation) =
    simulation.diagnostic_variables.grid.div_grid
SpeedyWeather.path(::HumidOutput, simulation) =
    simulation.diagnostic_variables.grid.humid_grid
#SpeedyWeather.path(::PresOutput, simulation) =
    #simulation.diagnostic_variables.physics.cos_zenith

time_stepping = Leapfrog(spectral_grid, Δt_at_T31=Minute(30))
ocean = ConstantOceanClimatology(spectral_grid)
planet = Earth(spectral_grid, seasonal_cycle = true, equinox = DateTime(2000, 3, 20))
convection = SimplifiedBettsMiller(spectral_grid)
large_scale_condensation = ImplicitCondensation(spectral_grid)
temperature_relaxation = JablonowskiRelaxation(spectral_grid)
model = PrimitiveWetModel(spectral_grid; output, time_stepping, temperature_relaxation, planet, ocean)
orography = EarthOrography(spectral_grid)
initialize!(orography, model) 
simulation = initialize!(model)

In [ ]:
initialize!(simulation.prognostic_variables, StartFromFile(path=LAYER_FOLDER), model)

### Find Perturbation

In [3]:
using NPZ
using NCDatasets
using CairoMakie

data_npz = npzread(REANALYSIS_NPZ)
ds = NCDataset(OUTPUT_FOLDER_PRE*OG_REANALYSIS_NC)

#length of time
start_t = 29221 # 2021-01-01 start 
finish_t = start_t + 365*4*15 #15 years
all_time = ds["time"][:]
times = ds["time"][start_t : finish_t]
time_len = size(times)[1]
lat_count = 48
lon_count = 96
year_steps = 1460

print("Is the start of the reanalysis ", start_t, " time steps away from 2001-01-01 ? \n")
print(data_npz["temperature"][1,:,:] == permutedims(ds["temp"][:,:,1,start_t], (2,1)))

print("\nHence (finish_t - 500) same as (end - 500) in reanalysis.npz will still be within the testing dataset\n")
print(data_npz["temperature"][end-500,:,:] == permutedims(ds["temp"][:,:,1,finish_t-500], (2,1)))

Is the start of the reanalysis 29221 time steps away from 2001-01-01 ? 
true
Hence (finish_t - 500) same as (end - 500) in reanalysis.npz will still be within the testing dataset
true

In [4]:
prog_variables = ["temperature", "humidity", "u_wind", "v_wind", "surface_pressure"]
prog_data_pert = Dict{String, Any}()
prog_pert_field = Dict{String, Any}()
for key in prog_variables
    prog_data_pert[key] = Float64.(data_npz[key][end - 500, :, :]) - Float64.(data_npz[key][end - 528, :, :])
    prog_pert_field[key] = FullGaussianGrid(prog_data_pert[key], input_as=Matrix)
end
typeof(prog_pert_field["temperature"])

Field{Float64, 1, Vector{Float64}, FullGaussianGrid{SpeedyWeather.Architectures.CPU{KernelAbstractions.CPU}, Vector{UnitRange{Int64}}, Vector{Int64}}}

In [15]:
using LinearAlgebra
S_fp64 = SpectralTransform(spectral_grid)
prog_variables = ["temperature", "humidity", "u_wind", "v_wind", "surface_pressure"]
pert_grid_to_spectral = Dict{String, Any}()
a = Dict{String, Any}()
b = Dict{String, Any}()
back_to_grid = Dict{String,Any}()

for key in prog_variables
    pert_grid_to_spectral[key] = transform(prog_pert_field[key], S_fp64)
    a[key] = transform(pert_grid_to_spectral[key], S_fp64)
    b[key] = transform(a[key], S_fp64)
    back_to_grid[key] = transform(b[key], S_fp64)
end

print(norm(prog_pert_field["temperature"]))
prog_pert_field["temperature"]-back_to_grid["temperature"]

1.161217491721248

4608-element, 48-ring FullGaussianField{Float64, 1} on Array on SpeedyWeather.Architectures.CPU{KernelAbstractions.CPU}:
  0.005172974129021307
  0.00512936744171262
  0.0076450153543531855
  0.009391623588408258
  0.008886753729962286
  0.00716523988126917
  0.005810881525153046
  0.006147844126365484
  0.006173789148704065
  0.0011093561260302753
 -0.008407694601436573
 -0.0150863509618537
 -0.015892228597319327
  ⋮
  0.004212512041440941
  0.003916724692638365
  0.003972208599149288
  0.004013000422847411
  0.0038254140653259305
  0.004019432813383041
  0.0038619739317129016
  0.002970634442666014
  0.0022139873800633716
  0.001972114536223251
  0.0018009538274961759
  0.0018361532246987861

In [6]:
rotate_map(x::Any) = permutedims(x, (1,2))

fig = Figure(size=(800, 1000))
ax1, hm1 = heatmap(fig[1,1], permutedims(rotate_map(prog_data_pert["temperature"])), colormap=:viridis)
ax1.yreversed=true
Colorbar(fig[1,2], hm1)

ax2, hm2 = heatmap(fig[2,1], permutedims(rotate_map(data_npz["temperature"][end - 500, :, :])), colormap=:viridis)
Colorbar(fig[2,2], hm2)
ax2.yreversed=true
# fig


true

In [ ]:
a = simulation.prognostic_variables.vor

560×1×2 LowerTriangularArray{ComplexF64, 3, Array{ComplexF64, 3}, Spectrum{SpeedyWeather.Architectures.CPU{KernelAbstractions.CPU}, Vector{UnitRange{Int64}}, Vector{Int64}}}
[:, :, 1] =
   -5.10702591327572e-15 + 0.0im
  -2.3283064365386963e-9 + 0.0im
   1.5459954738616943e-7 + 0.0im
   -8.288770914077759e-8 + 0.0im
    6.984919309616089e-8 + 0.0im
   -3.594905138015747e-7 + 0.0im
  -1.4435499906539917e-7 + 0.0im
   -6.295740604400635e-7 + 0.0im
   -5.327165126800537e-7 + 0.0im
   -7.189810276031494e-7 + 0.0im
   -3.650784492492676e-7 + 0.0im
  -4.2282044887542725e-7 + 0.0im
     -7.7858567237854e-7 + 0.0im
                         ⋮
  -1.0477378964424133e-9 - 9.74978320300579e-10im
   1.7898855730891228e-9 + 1.6007106751203537e-9im
  1.8917489796876907e-10 - 6.912159733474255e-11im
   -1.076841726899147e-9 - 1.760781742632389e-9im
   7.676135282963514e-10 - 2.08092387765646e-9im
 -2.9984903449076228e-12 + 1.2514647096395493e-9im
  -6.275513442233205e-11 + 5.2523319027386606e-11im
    

In [29]:
set!(simulation.prognostic_variables, model.geometry; u = prog_pert_field["u_wind"], add=true)

In [31]:
a - simulation.prognostic_variables.vor

560×1×2 LowerTriangularArray{ComplexF64, 3, Array{ComplexF64, 3}, Spectrum{SpeedyWeather.Architectures.CPU{KernelAbstractions.CPU}, Vector{UnitRange{Int64}}, Vector{Int64}}}
[:, :, 1] =
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
     ⋮
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

[:, :, 2] =
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
     ⋮
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im